#1. Corpus
#Chinese

In [ ]:
! pip install requests beautifulsoup4


In [ ]:
! pip install tqdm


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import csv
from tqdm import tqdm  # 新增导入

start_date = datetime(2015, 1, 1)
end_date = datetime(2025, 7, 1)

headers = {
    'User-Agent': 'Mozilla/5.0'
}

articles_data = []

#  开始计时
start_time = time.time()

#  构建日期列表供 tqdm 使用
total_days = (end_date - start_date).days + 1
date_list = [start_date + timedelta(days=i) for i in range(total_days)]

#  使用 tqdm 包装 date_list 以显示进度条
for current_date in tqdm(date_list, desc="抓取进度"):
    date_str = current_date.strftime("%Y-%m/%d")
    base_url = f"http://paper.people.com.cn/rmrb/html/{date_str}/nbs.D110000renmrb_01.htm"

    try:
        res = requests.get(base_url, headers=headers, timeout=10)
        res.encoding = 'utf-8'
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'html.parser')
            page_links = soup.select("div#pageList a")
            for page in page_links:
                page_href = page.get('href')
                if not page_href:
                    continue
                page_url = f"http://paper.people.com.cn/rmrb/html/{date_str}/{page_href}"
                try:
                    page_res = requests.get(page_url, headers=headers, timeout=10)
                    page_res.encoding = 'utf-8'
                    page_soup = BeautifulSoup(page_res.text, 'html.parser')
                    article_links = page_soup.select("div.news a")
                    for a in article_links:
                        href = a.get('href')
                        if not href:
                            continue
                        article_url = f"http://paper.people.com.cn/rmrb/html/{date_str}/{href}"
                        try:
                            article_res = requests.get(article_url, headers=headers, timeout=10)
                            article_res.encoding = 'utf-8'
                            article_soup = BeautifulSoup(article_res.text, 'html.parser')
                            title = article_soup.select_one("h1").get_text(strip=True) if article_soup.select_one("h1") else ""
                            subtitle = article_soup.select_one("h2").get_text(strip=True) if article_soup.select_one("h2") else ""
                            content_div = article_soup.select_one("div#ozoom")
                            content = content_div.get_text(separator="\n").strip() if content_div else ""
                            articles_data.append([
                                current_date.strftime("%Y-%m-%d"),
                                title,
                                subtitle,
                                content,
                                article_url
                            ])
                        except Exception:
                            continue
                except Exception:
                    continue
        time.sleep(1)  # 每天请求间隔1秒，避免被封IP
    except Exception:
        pass

# 保存为CSV文件
with open("people_daily_2015_2025.csv", "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["日期", "标题", "副标题", "正文", "网址"])
    writer.writerows(articles_data)

print(" 抓取完成，共收集文章数：", len(articles_data))

# ⏱ 显示总耗时
end_time = time.time()
total_time = end_time - start_time
minutes = int(total_time // 60)
seconds = int(total_time % 60)
print(f"⏱ 总耗时：{minutes} 分 {seconds} 秒")


抓取进度:   9%|▉         | 362/3835 [07:47<1:14:47,  1.29s/it]


KeyboardInterrupt: 

#1) Quotidien du Peuple

2022-2025
http://paper.people.com.cn/rmrb/pc/layout/202507/23/node_01.html


#2) https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2


# 3) http://www.people.com.cn/


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 目标网页
url = "https://paper.people.com.cn/rmrb/html/2024-07/01/nw.D110000renmrb_20240701_6-20.htm"
headers = {
    'User-Agent': 'Mozilla/5.0'
}

# 发起请求
res = requests.get(url, headers=headers)
res.encoding = 'utf-8'

if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')

    # 提取字段
    title = soup.select_one("h1").get_text(strip=True) if soup.select_one("h1") else ""
    subtitle = soup.select_one("h2").get_text(strip=True) if soup.select_one("h2") else ""
    title = f"{title}: {subtitle}" if subtitle else title

    # 作者可能在 <div class="edit">，有时在正文里
    author_tag = soup.select_one("div.edit")
    author = author_tag.get_text(strip=True).replace("编辑：", "") if author_tag else ""

    # 日期从 URL 获取
    date = "2022-07-01"

    # 正文
    content_div = soup.select_one("div#ozoom")
    content = content_div.get_text(separator="\n").strip() if content_div else ""

    # 来源（固定）
    source = "People's Daily (Renmin Ribao)"

    # 写入 CSV 文件
    with open("quotidienne87.csv", "w", encoding="utf-8-sig", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["author", "title", "date", "content", "source"])
        writer.writerow([author, title, date, content, source])

    print(" 成功保存为 'quotidienne6.csv'")
else:
    print(f" 请求失败，状态码：{res.status_code}")


SyntaxError: unterminated string literal (detected at line 6) (ipython-input-614328957.py, line 6)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标 URL
url = "http://www.people.com.cn/n1/2015/1214/c393876-27927336.html"

# 请求网页
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = "utf-8"  # 确保中文不会乱码

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# ===== 提取标题 =====
title_tag = soup.find("h1", id="p_title")
title = title_tag.get_text(strip=True) if title_tag else ""

# ===== 提取作者 =====
author_tag = soup.find("i", id="p_editor")
author = author_tag.get_text(strip=True) if author_tag else ""

# ===== 提取日期 =====
date_tag = soup.find("span", id="p_publishtime")
date = date_tag.get_text(strip=True) if date_tag else ""

# ===== 提取正文内容 =====
content_div = soup.find("div", id="p_content")
paragraphs = content_div.find_all("p") if content_div else []
content = "\n".join(p.get_text(strip=True) for p in paragraphs)

# ===== 保存数据到 CSV =====
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": url
}]

df = pd.DataFrame(data)
df.to_csv("peuple2015_6.csv", index=False, encoding="utf-8-sig")

print("已完成抓取，已保存为 peuple2016.csv")


已完成抓取，已保存为 peuple2016.csv


In [ ]:
#2016


import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标 URL
url = "http://www.people.com.cn/n1/2016/0901/c397545-28684604.html"

# 请求网页
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = "utf-8"  # 确保中文不会乱码

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# ===== 提取标题 =====
title_tag = soup.find("h1")
title = title_tag.get_text(strip=True) if title_tag else ""

# ===== 提取日期 =====
# 日期在 <div class="fl"> 里
date_tag = soup.find("div", class_="fl")
date = date_tag.get_text(strip=True) if date_tag else ""

# ===== 提取作者 =====
# 作者在 <div class="edit clearfix"> 里
author_tag = soup.find("div", class_="edit clearfix")
author = author_tag.get_text(strip=True) if author_tag else ""

# ===== 提取正文内容 =====
content_div = soup.find("div", id="rwb_zw")
content = ""

if content_div:
    # 只取所有 <p> 标签的文本（过滤掉图片说明和空标签）
    paragraphs = content_div.find_all("p")
    content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

# ===== 保存到 CSV =====
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": url
}]

df = pd.DataFrame(data)
df.to_csv("peuple2016_17.csv", index=False, encoding="utf-8-sig")

print(" 已完成抓取，已保存为 peuple2016.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网址
url = "http://www.people.com.cn/n/2015/0921/c347079-27612558.html"

# 发送请求并自动检测编码
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding  # 自动识别编码，防止乱码

# 用 BeautifulSoup 解析
soup = BeautifulSoup(response.text, "html.parser")

# =========  提取标题 =========
title = ""
title_tag = soup.find("h1", id="p_title")
if title_tag:
    title = title_tag.get_text(strip=True)

# ========= 提取日期 =========
date = ""
date_tag = soup.find("span", id="p_publishtime")
if date_tag:
    date = date_tag.get_text(strip=True)

# ========= 提取作者 =========
author = ""
author_tag = soup.find("div", class_="edit")  # 注意这里有时是 class="edit clearfix"
if author_tag:
    author = author_tag.get_text(strip=True)

# ========= 提取正文内容 =========
content = ""
content_div = soup.find("div", id="p_content")
if content_div:
    paragraphs = content_div.find_all("p")
    content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

# ========= 保存到 CSV =========
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": url
}]

df = pd.DataFrame(data)
df.to_csv("peuple2015_23.csv", index=False, encoding="utf-8-sig")  # 用 utf-8-sig 确保 Excel 打开不乱码

# 打印结果验证
print(f"抓取完成！\n标题: {title}\n日期: {date}\n作者: {author}\n内容预览: {content[:60]}...")


抓取完成！
标题: 政务微博办实事排行榜59：@中国保险报 获赞最多
日期: 2015年09月21日09:27
作者: (责编：张娅喃、李小炜)
内容预览: 人民网北京9月21日电（张娅喃 实习生熊捷）9月21日，在人民微博“对话官微”推出的《全国政务微博办实事排行榜》第59期...


In [ ]:
#2018

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网址
url = "http://www.people.com.cn/n1/2018/0303/c32306-29845828.html"

# 发送请求并自动检测编码
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding  # 自动识别编码，防止乱码

# 用 BeautifulSoup 解析
soup = BeautifulSoup(response.text, "html.parser")

# =========  提取标题 =========
title = ""
title_tag = soup.find("h1", id="p_title")
if title_tag:
    title = title_tag.get_text(strip=True)

# ========= 提取日期 =========
date = ""
date_tag = soup.find("i", id="p_publishtime")   #  这里改成<i>，不是<span>
if date_tag:
    date = date_tag.get_text(strip=True)

# ========= 提取作者 =========
author = ""
author_tag = soup.find("div", id="p_editor")   # 人民网作者标签是id="p_editor"
if author_tag:
    author = author_tag.get_text(strip=True).replace("(责编：", "").replace(")", "")

# ========= 提取正文内容 =========
content = ""
content_div = soup.find("div", id="p_content")
if content_div:
    paragraphs = content_div.find_all("p")
    content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

# ========= 保存到 CSV =========
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "source": url
}]

df = pd.DataFrame(data)
df.to_csv("peuple2018_4.csv", index=False, encoding="utf-8-sig")  # 用 utf-8-sig，Excel 打开不乱码

# 打印结果验证
print(f"抓取完成！\n标题: {title}\n日期: {date}\n作者: {author}\n内容预览: {content[:60]}...")



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网址
url = "http://www.people.com.cn/n1/2019/0304/c32306-30956836.html"

# 请求头（模拟浏览器，避免被拦截或返回跳转页）
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

# 发送请求
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding  # 自动识别编码，防止乱码

# 🛠️ 调试：打印 URL 和 HTML 前 1000 个字符，检查是否拿到正确页面
print("实际抓取的 URL:", response.url)
print("HTML 前 1000 字符:\n", response.text[:1000])

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# ========= 提取标题 =========
title_tag = soup.find("h1", id="p_title") or soup.find("h1")  # 优先找 id="p_title"，否则找 h1
title = title_tag.get_text(strip=True) if title_tag else ""
print("标题:", title)

# ========= 提取日期 =========
date_tag = soup.find("div", class_="col-1-1 fl")
date = date_tag.get_text(strip=True).split("|")[0] if date_tag else ""
print("日期:", date)

# ========= 提取作者 =========
author_tag = soup.find("div", class_="edit cf")
author = author_tag.get_text(strip=True).replace("(责编：", "").replace(")", "") if author_tag else ""
print("作者:", author)

# ========= 提取正文 =========
content = ""
content_div = soup.find("div", class_="rm_txt_con cf")
if content_div:
    paragraphs = content_div.find_all("p")
    content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
print("正文前 60 字:", content[:60])

# ========= 保存到 CSV =========
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "source": url
}]

df = pd.DataFrame(data)
df.to_csv("peuple2019_18.csv", index=False, encoding="utf-8-sig")  # 用 utf-8-sig，Excel 打开不乱码

print("抓取完成！已生成 peuple2019.csv ")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网址
url = "http://www.people.com.cn/n1/2019/0306/c425620-30960965.html"

# 请求头（模拟浏览器）
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

# 发送请求
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding  # 自动识别编码，避免乱码

# 🛠️ 调试输出（可注释）
print("实际抓取的 URL:", response.url)
print("HTML 前 500 字符:\n", response.text[:500])

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# ========= 提取标题 =========
title_tag = soup.find("h1", id="p_title")
title = title_tag.get_text(strip=True) if title_tag else ""
print("标题:", title)

# ========= 提取日期 =========
date_tag = soup.find("i", id="p_publishtime")
date = date_tag.get_text(strip=True) if date_tag else ""
print("日期:", date)

# ========= 提取作者 =========
author_tag = soup.find("div", id="p_editor")
author = author_tag.get_text(strip=True).replace("(责编：", "").replace(")", "") if author_tag else ""
print("作者:", author)

# ========= 提取正文 =========
content = ""
content_div = soup.find("div", id="p_content")
if content_div:
    paragraphs = content_div.find_all("p")
    content = "\n".join(
        p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)
    )
print("正文前 60 字:", content[:60])

# ========= 保存到 CSV =========
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "source": url
}]

df = pd.DataFrame(data)
df.to_csv("peuple2019_20.csv", index=False, encoding="utf-8-sig")  # 用 utf-8-sig，Excel 打开不乱码

print(" 抓取完成！已生成 peuple2019.csv ")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 目标网址
url = "http://world.people.com.cn/n1/2024/0206/c1002-40174467.html"

# 请求网页
response = requests.get(url)
response.encoding = response.apparent_encoding  # 自动检测真实编码
html = response.text

# 解析 HTML
soup = BeautifulSoup(html, "html.parser")

# === 抓取标题 ===

title = "摩尔多瓦首发龙年生肖邮票"

# === 抓取作者 (责编) ===
# 注意：有时会有多个 edit cf，正文底部的才是作者
author_divs = soup.find_all("div", class_="edit cf")
author = author_divs[-1].get_text(strip=True).replace("责编：", "") if author_divs else ""

# ===  抓取日期和来源 ===
date_div = soup.find("div", class_="col-1-1 fl")
date_text = ""
source = ""
if date_div:
    date_text = date_div.get_text(strip=True).split("|")[0].strip()
    source_a = date_div.find("a")
    source = source_a.get_text(strip=True) if source_a else ""

# === 抓取正文内容 ===
content_div = soup.find("div", class_="rm_txt_con cf")
paragraphs = []
if content_div:
    for p in content_div.find_all("p"):
        txt = p.get_text(strip=True)
        if txt:
            paragraphs.append(txt)
content = "\n".join(paragraphs)

# === 保存到 CSV ===
with open("peuple2024_1.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["author", "title", "date", "content", "source"])
    writer.writerow([author, title, date_text, content, source])

print("抓取完成，已保存到 peuple2023_1.csv")
print(f"标题：{title}")
print(f"日期：{date_text}")
print(f"来源：{source}")
print(f"作者：{author}")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/peuple_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("peuple2015_"):
            zipf.write(os.path.join("/content", file), arcname=file)


In [ ]:
from google.colab import files
files.download("/content/peuple_all_csv.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#penbaixinwei


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 设置 URL
url = "https://www.thepaper.cn/newsDetail_forward_31158236"

# 请求头
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 获取网页内容
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

# 提取标题
title_tag = soup.find("h1", class_="index_title__B8mhI")
title = title_tag.get_text(strip=True) if title_tag else ""

# 提取作者（通常在标题上方或下方的某个 <div> 中，包含“澎湃新闻记者”）
author = ""
for div in soup.find_all("div"):
    if div.text.strip().startswith("澎湃新闻记者"):
        author = div.text.strip().replace("澎湃新闻记者", "").strip()
        break

# 提取日期（第一个包含时间格式的 <span>）
date = ""
for span in soup.find_all("span"):
    if span.text.strip().startswith("20") and ":" in span.text:
        date = span.text.strip()
        break

# 提取正文内容
content_div = soup.find("div", class_="index_cententWrap__Jv8jK")
paragraphs = content_div.find_all("p") if content_div else []
content = "\n".join([p.get_text(strip=True) for p in paragraphs])

# 构造数据
data = [[author, title, date, content, url]]
df = pd.DataFrame(data, columns=["author", "title", "date", "content", "source"])

# 保存为 CSV
df.to_csv("pengpai2020_140.csv", index=False, encoding="utf-8-sig")
print(" 抓取完成，保存 pengpai2015_1.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 目标 URL
url = "https://www.thepaper.cn/newsDetail_forward_1351726"

# 发送请求
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
response.encoding = "utf-8"  # 澎湃新闻用utf-8编码

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# 提取 author
author_tag = soup.find("div", string=lambda t: t and "澎湃新闻记者" in t)
author = author_tag.get_text(strip=True) if author_tag else ""

# 提取 title
title_tag = soup.find("h1", class_="index_title__B8mhI")
title = title_tag.get_text(strip=True) if title_tag else ""

# 提取 date
date_tag = soup.find("span")
date = date_tag.get_text(strip=True) if date_tag else ""

# 提取 content
content_div = soup.find("div", class_="index_cententWrap__Jv8jK")
if content_div:
    # 去掉多余标签，只保留纯文本
    content = "\n".join(p.get_text(strip=True) for p in content_div.find_all("p"))
else:
    content = ""

# 保存到 CSV
csv_filename = "pengpai2015_5.csv"
with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["author", "title", "date", "content", "url"])
    writer.writerow([author, title, date, content, url])

print(f"数据已保存到 {csv_filename}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 目标网页
url = "https://www.thepaper.cn/newsDetail_forward_8250801"

# 请求网页
response = requests.get(url)
response.encoding = "utf-8"
soup = BeautifulSoup(response.text, "html.parser")

# 1️标题
title_tag = soup.find("h1", class_="index_title__B8mhI")
title = title_tag.get_text(strip=True) if title_tag else ""

# 2️日期（通常是第一个 span，没有“来源”字样）
date_tag = soup.find("span", string=lambda t: t and "来源" not in t)
date = date_tag.get_text(strip=True) if date_tag else ""

# 3️作者（查找包含“来源”的 span，并去掉前缀“来源：”）
author_tag = soup.find("span", string=lambda t: t and "来源" in t)
author = author_tag.get_text(strip=True).replace("来源：", "") if author_tag else ""

# 4️正文（所有 <p> 里的文字）
content_div = soup.find("div", class_="index_cententWrap__Jv8jK")
content = ""
if content_div:
    paragraphs = content_div.find_all("p")
    content = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])

# 5️写入 CSV
csv_file = "pengpai2020_19.csv"
with open(csv_file, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["author", "title", "date", "content", "url"])
    writer.writeheader()
    writer.writerow({
        "author": author,
        "title": title,
        "date": date,
        "content": content,
        "url": url
    })

print(f"抓取完成，已保存到 {csv_file}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 要抓取的网页
url = "https://www.thepaper.cn/newsDetail_forward_29761657"

# 请求网页
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
response.encoding = "utf-8"

# 用 BeautifulSoup 解析
soup = BeautifulSoup(response.text, "html.parser")

# ===== 抓取标题 =====
title_tag = soup.find("h1", class_="index_title__B8mhI")
title = title_tag.get_text(strip=True) if title_tag else ""

# ===== 抓取作者和日期 =====
# 获取页面中所有 span 标签（顺序敏感）
spans = soup.find_all("span")

author = ""
date = ""

# 遍历 span 标签，找到包含“>”的作为作者，包含“-”的作为日期
for s in spans:
    text = s.get_text(strip=True)
    if ">" in text and author == "":
        author = text
    elif "-" in text and date == "":
        date = text

# ===== 抓取正文内容 =====
content_div = soup.find("div", class_="index_cententWrap__Jv8jK")
paragraphs = content_div.find_all("p") if content_div else []
content = "\n".join(p.get_text(strip=True) for p in paragraphs)

# ===== 生成 CSV =====
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": url
}]

df = pd.DataFrame(data)
df.to_csv("pengpai2024_21.csv", index=False, encoding="utf-8-sig")

print("已完成抓取，已保存为 pengpai2024.csv")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/pengpai_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("pengpai2024_"):
            zipf.write(os.path.join("/content", file), arcname=file)


In [ ]:
from google.colab import files
files.download("/content/pengpai_all_csv.zip")


#3 彭博财经
www.bloomberg.com


#https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2

#4） 网易新闻
#news.163.com


#5）新浪新闻网
# news.sina.com.cn


#6) 新华网、东方网

#http://www.xinhuanet.com

#http://www.eastday.com



In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网页URL
url = "http://www.xinhuanet.com/politics/2020-07/26/c_1126287151.htm"

# 抓取并解析
resp = requests.get(url, timeout=15)
resp.encoding = "utf-8"
soup = BeautifulSoup(resp.text, "html.parser")

# 标题
title_tag = soup.select_one("div.h-title")
title = title_tag.get_text(strip=True) if title_tag else ""

# 日期
date_tag = soup.select_one("span.h-time")
date = date_tag.get_text(strip=True) if date_tag else ""

# 作者（责任编辑）
author = ""
pjc = soup.select_one("span.p-jc")
if pjc:
    raw = pjc.get_text(" ", strip=True)
    m = re.search(r"责任编辑[:：]\s*([^\s\]]+)", raw)
    author = m.group(1) if m else raw

# 正文
content = ""
content_div = soup.find("div", id="p-detail")
if content_div:
    # 去掉视频/脚本等无关块
    for tag in content_div.find_all(["script", "style", "iframe"]):
        tag.decompose()
    # 取出段落文字（含加粗段）
    parts = []
    for tag in content_div.find_all(["p", "strong"]):
        txt = tag.get_text(" ", strip=True)
        if txt:
            parts.append(txt)
    content = "\n".join(parts).strip()

# 组装并保存
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": url
}]
df = pd.DataFrame(data)

outname = "xinhua2020_22.csv"
df.to_csv(outname, index=False, encoding="utf-8-sig")
print(f"文件已保存为 {outname}")


In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

def normalize_space(s: str) -> str:
    if not s:
        return ""
    # 去掉全角空格/不间断空格，压缩多余空白
    s = s.replace("\u3000", " ").replace("\xa0", " ")
    s = re.sub(r"[ \t]+", " ", s)
    # 去掉标点前的多余空格
    s = re.sub(r"\s+([，。、“”‘’；：！？,.!?;:])", r"\1", s)
    return s.strip()

# 你的目标页
url = "http://www.xinhuanet.com/world/20250527/a7997efbf96b4fe0a5e3e5731b136790/c.html"

# 抓取页面
resp = requests.get(url, timeout=20, headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
})
# 新华网多为 UTF-8，这里使用 apparent_encoding 兜底
resp.encoding = resp.apparent_encoding or "utf-8"
soup = BeautifulSoup(resp.text, "html.parser")

# 标题：<span class="title">
title = ""
title_tag = soup.select_one("span.title")
if title_tag:
    title = normalize_space(title_tag.get_text(strip=True))

# 日期：<div class="header-time left"><span class="year"><em> 2021</em></span><span class="day">02/16</span><span class="time"> 14:32:57</span>
year = soup.select_one("div.header-time .year em")
day = soup.select_one("div.header-time .day")
tm  = soup.select_one("div.header-time .time")
date = ""
if year and day and tm:
    y = normalize_space(year.get_text())
    d = normalize_space(day.get_text()).replace("/", "-")  # 02/16 -> 02-16
    t = normalize_space(tm.get_text())
    date = f"{y}-{d} {t}"

# 作者（责任编辑）：通常在 <span class="editor">【责任编辑:王秀】</span>
author = ""
# 直接找所有 editor，拼出文本再用正则提取
editors = soup.select("span.editor")
if editors:
    raw = " ".join(normalize_space(e.get_text(" ", strip=True)) for e in editors)
    m = re.search(r"责任编辑[:：]?\s*([^\s】\]]+)", raw)
    if m:
        author = m.group(1)
# 兜底：全页找“责任编辑”
if not author:
    node = soup.find(string=re.compile(r"责任编辑"))
    if node:
        txt = normalize_space(str(node))
        m2 = re.search(r"责任编辑[:：]?\s*([^\s】\]]+)", txt)
        if m2:
            author = m2.group(1)

# 正文：<div id="detail"> 下的 <p>；去掉 articleEdit 等杂项
content = ""
detail = soup.find("div", id="detail")
if detail:
    # 去掉无关块（纠错、iframe 等）
    for bad in detail.find_all(["script", "style", "iframe"]):
        bad.decompose()
    ae = detail.find("div", id="articleEdit")
    if ae:
        ae.decompose()

    parts = []
    for p in detail.find_all("p"):
        # 跳过只含图片/空行的段落
        if p.find(["img", "iframe"]):
            continue
        txt = normalize_space(p.get_text(" ", strip=True))
        if not txt:
            continue
        # 过滤“纠错”等噪声
        if "纠错" in txt or "点击进入专题" in txt:
            continue
        parts.append(txt)

    # 去重并合并
    seen = set()
    dedup = []
    for t in parts:
        if t not in seen:
            seen.add(t)
            dedup.append(t)
    content = "\n".join(dedup).strip()

# 组装并保存
data = [{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": url
}]
df = pd.DataFrame(data)

outname = "xinhua2025_26.csv"
df.to_csv(outname, index=False, encoding="utf-8-sig")
print(f"文件已保存为 {outname}")
print(df.T)


文件已保存为 xinhua2025_26.csv
                                                         0
author                                                 张艳芳
title                    全球连线｜2025中德（欧）隐形冠军论坛开幕 共享经济技术合作机遇
date                                   2025-05-27 14:13:43
content  5月26日，北京中德产业合作发展论坛——2025中德（欧）隐形冠军论坛在北京顺义中德国际会议...
url      http://www.xinhuanet.com/world/20250527/a7997e...


In [ ]:
#另一种简单的字

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网页URL
url = "http://www.xinhuanet.com/world/2021-03/24/c_1127250212.htm"

# 发送HTTP请求获取网页内容
response = requests.get(url)
response.encoding = 'utf-8'  # 确保中文编码正确

# 使用BeautifulSoup解析网页内容
soup = BeautifulSoup(response.text, 'html.parser')

# 提取标题
title = soup.find('h1', id='title').get_text(strip=True)

# 提取日期
date = soup.find('span', id='pubtime').get_text(strip=True)

# 提取内容
content_div = soup.find('div', id='content')
content = "\n".join([p.get_text(strip=True) for p in content_div.find_all('p')])

# 提取作者
author_div = soup.find('div', class_='share')
author = author_div.get_text(strip=True).replace("[责任编辑：", "").replace("]", "") if author_div else ""

# URL
article_url = url

# 将数据存储为字典
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content],
    "url": [article_url]
}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 保存为CSV文件
df.to_csv('xinhua2020_1.csv', index=False, encoding='utf-8-sig')

print("文件已保存为 xinhua2015_1.csv")


In [ ]:
#code 3

import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

# 目标网页URL（首篇）
url = "http://www.xinhuanet.com/world/2021-03/24/c_1127250212.htm"

def fetch(url):
    r = requests.get(url, timeout=15)
    r.encoding = 'utf-8'  # 新华网老文基本是 utf-8
    return BeautifulSoup(r.text, 'html.parser')

soup = fetch(url)

# 标题
title = soup.find('h1', id='title').get_text(strip=True)

# 日期
date = soup.find('span', class_='time').get_text(strip=True)

# 作者（责任编辑）
editor = soup.find('span', class_='editor')
author = ""
if editor:
    author = re.sub(r"[\[\]：:\s]*", "", editor.get_text())
    author = author.replace("责任编辑", "")  # 只保留名字

# --- 正文（含分页） ---
def collect_content(soup, base_url):
    # 本页正文
    article = soup.find('div', class_='article')
    parts = []
    if article:
        # 把 p 和 strong 的文字都取出来
        for tag in article.find_all(['p', 'strong']):
            txt = tag.get_text(" ", strip=True)
            if txt:
                parts.append(txt)

        # 找到分页的链接（如 _2.htm），并抓取
        page_links = set()
        for a in article.find_all('a', href=True):
            href = a['href']
            if re.search(r"_\d+\.htm$", href):  # 只要分页页
                full = urljoin(base_url if base_url.startswith('http') else "http:", href)
                page_links.add(full)

        # 按序抓取其他页
        for link in sorted(page_links):
            sub = fetch(link)
            sub_article = sub.find('div', class_='article')
            if sub_article:
                for tag in sub_article.find_all(['p', 'strong']):
                    txt = tag.get_text(" ", strip=True)
                    if txt:
                        parts.append(txt)

    # 合并成一段文本
    content = "\n".join(parts)
    # 轻度清洗：去掉多余空白
    content = re.sub(r"\s+\n", "\n", content).strip()
    return content

content = collect_content(soup, url)

# URL
article_url = url

# 组装DataFrame
df = pd.DataFrame([{
    "author": author,
    "title": title,
    "date": date,
    "content": content,
    "url": article_url
}])

# 保存CSV
outname = 'xinhua2020_1.csv'
df.to_csv(outname, index=False, encoding='utf-8-sig')
print(f"文件已保存为 {outname}")



In [ ]:
#code 4 : nous ne parvenons pas

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网页URL
url = "http://www.xinhuanet.com/2021-02/16/c_1127105025.htm"

# 发送HTTP请求获取网页内容
response = requests.get(url)
response.encoding = 'utf-8'  # 防止中文乱码

# 使用BeautifulSoup解析网页内容
soup = BeautifulSoup(response.text, 'html.parser')

# 提取标题
title = soup.find('h1', id='title').get_text(strip=True) if soup.find('h1', id='title') else ''

# 提取日期
date = soup.find('span', class_='time').get_text(strip=True) if soup.find('span', class_='time') else ''

# 提取作者
author_tag = soup.find('span', class_='editor')
author = author_tag.get_text(strip=True).replace('[责任编辑:', '').replace(']', '') if author_tag else ''

# 提取正文内容
content_div = soup.find('div', class_='article')
content = ""
if content_div:
    paragraphs = content_div.find_all('p')
    content = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])

# URL
article_url = url

# 将数据存储为字典
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content],
    "url": [article_url]
}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 保存为CSV文件
df.to_csv('xinhua2020_9.csv', index=False, encoding='utf-8-sig')

print("文件已保存为 xinhua_2017.csv")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/xinhua_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("xinhua2025_"):
            zipf.write(os.path.join("/content", file), arcname=file)


In [ ]:
from google.colab import files
files.download("/content/xinhua_all_csv.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#3） 南方周末
#https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2

# https://www.infzm.com/

In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/nanfang_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("nanfang_"):
            zipf.write(os.path.join("/content", file), arcname=file)


In [ ]:
from google.colab import files
files.download("/content/pengpai_all_csv.zip")


# 3) ZHIHU

In [ ]:
! pip install selenium beautifulsoup4 pandas


In [ ]:
#zhihu  话题 2015

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置关键词和搜索地址
search_keyword = "2015"
search_url = f"https://www.zhihu.com/search?q={search_keyword}&type=content"

# 配置浏览器
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)
driver.get(search_url)
time.sleep(5)

# 模拟滚动加载更多话题卡片
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 解析页面
soup = BeautifulSoup(driver.page_source, "html.parser")
topic_cards = soup.find_all("a", class_="TopicLink")

data = []

for topic in topic_cards:
    try:
        topic_name = topic.get_text(strip=True)
        href = topic.get("href")
        full_url = "https://www.zhihu.com" + href if href.startswith("/") else href

        # 向上寻找卡片父元素，提取简介和浏览数
        parent_div = topic.find_parent("div", class_="ContentItem-head")
        description = ""
        views = ""
        discussions = ""

        if parent_div:
            meta_div = parent_div.find_next("div", class_="ContentItem-meta")
            if meta_div:
                desc_div = meta_div.find("div", class_="ztext")
                status_links = meta_div.find_all("a", class_="Search-statusLink")
                if desc_div:
                    description = desc_div.get_text(strip=True)
                if status_links and len(status_links) >= 2:
                    views = status_links[0].get_text(strip=True)
                    discussions = status_links[1].get_text(strip=True)

        data.append([topic_name, description, views, discussions, full_url])
        print(f"抓取话题：{topic_name}")

    except Exception as e:
        print("忽略话题：", e)
        continue

driver.quit()

# 保存为 CSV
df = pd.DataFrame(data, columns=["topic", "description", "views", "discussions", "url"])
df.to_csv("zhihu_topics_2015.csv", index=False, encoding="utf-8-sig")
print(f"共保存 {len(data)} 个话题到 zhihu_topics_2015.csv")



In [ ]:
#每个页面的话题

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置关键词和搜索地址
search_keyword = "2015"
search_url = f"https://www.zhihu.com/search?q={search_keyword}&type=content"

# 配置 Selenium 无头浏览器
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0")

driver = webdriver.Chrome(options=options)
driver.get(search_url)
time.sleep(5)

# 滚动加载更多结果
for _ in range(15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 解析页面
soup = BeautifulSoup(driver.page_source, "html.parser")
topic_cards = soup.find_all("a", class_="TopicLink")

data = []

for topic in topic_cards:
    try:
        # 标题
        topic_name = topic.get_text(strip=True)

        # 链接
        href = topic.get("href")
        full_url = "https://www.zhihu.com" + href if href.startswith("/") else href

        # 找简介和数据
        parent_div = topic.find_parent("div", class_="ContentItem-head")
        description = ""
        views = ""
        discussions = ""

        if parent_div:
            meta_div = parent_div.find_next("div", class_="ContentItem-meta")
            if meta_div:
                desc_div = meta_div.find("div", class_="ztext")
                status_links = meta_div.find_all("a", class_="Search-statusLink")
                if desc_div:
                    description = desc_div.get_text(strip=True)
                if status_links and len(status_links) >= 2:
                    views = status_links[0].get_text(strip=True)
                    discussions = status_links[1].get_text(strip=True)

        data.append([topic_name, description, views, discussions, full_url])
        print(f"✔ 抓取话题：{topic_name}")

    except Exception as e:
        print(" 忽略话题：", e)
        continue

driver.quit()

# 保存为 CSV
df = pd.DataFrame(data, columns=["topic", "description", "views", "discussions", "url"])
df.to_csv("zhihu_topics_20151.csv", index=False, encoding="utf-8-sig")
print(f"抓取完成，共保存 {len(data)} 个话题到 zhihu_topics_2015.csv")


In [ ]:
#zhihu 综合 2015


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# 设置关键词和搜索类型为综合内容
search_keyword = "2015"
search_url = f"https://www.zhihu.com/search?q={search_keyword}&type=content"

# 配置 Selenium 浏览器
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)
driver.get(search_url)
time.sleep(5)

# 模拟下滑加载更多内容
for _ in range(15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 解析页面
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.find_all("div", class_="SearchResult-Card")

data = []

for card in cards:
    try:
        # 链接
        link_tag = card.find("a", href=True)
        if not link_tag:
            continue
        href = link_tag["href"]
        url = "https://www.zhihu.com" + href if href.startswith("/question") else href

        # 标题
        title = link_tag.get_text(strip=True)

        # 摘要
        summary_tag = card.find("div", class_="RichContent-inner")
        summary = summary_tag.get_text(strip=True) if summary_tag else ""

        # 作者
        author_tag = card.find("div", class_="AuthorInfo-head")
        author = author_tag.get_text(strip=True) if author_tag else "匿名"

        # 时间（尝试提取）
        time_tag = card.find("time")
        date_str = time_tag["datetime"][:10] if time_tag and "datetime" in time_tag.attrs else ""
        try:
            date = datetime.strptime(date_str, "%Y-%m-%d") if date_str else None
        except:
            date = None

        # 若未提取到时间，但关键词中明确包含“2015”，也可保留
        if date is None and "2015" in summary:
            date = datetime(2015, 1, 1)

        if date and datetime(2015, 1, 1) <= date <= datetime(2025, 12, 31):
            data.append([title, author, date.strftime("%Y-%m-%d"), summary, url])
            print("✔ 抓取：", title)

    except Exception as e:
        print("忽略内容：", e)
        continue

driver.quit()

# 保存为 CSV 文件
df = pd.DataFrame(data, columns=["title", "author", "date", "summary", "link"])
df.to_csv("zhihu_mixed_2015_2025.csv", index=False, encoding="utf-8-sig")
print(f"完成，共保存 {len(data)} 条内容到 zhihu_mixed_2015_2025.csv")


#彭博新闻

#4) Weibo

#5)Douban

#6)blibli - vidéo

#7baidu 百度新闻 ： https://www.baidu.com/s?tn=news&word=2015&pn=100



#查询： https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2


# **Corpus en vietnamien**

#1)https://kenh14.vn




In [ ]:
! pip install requests beautifulsoup4 pandas


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 目标网址
url = 'https://kenh14.vn/chum-anh-can-canh-rapper-binh-gold-thoi-diem-bi-csgt-truy-bat-tren-cao-toc-215250724113851073.chn'

headers = {
    'User-Agent': 'Mozilla/5.0'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# 1. 标题
title = soup.find('h1', class_='kbwc-title')
title_text = title.get_text(strip=True) if title else 'N/A'

# 2. 日期
date = soup.find('span', class_='kbwcm-time')
date_text = date.get_text(strip=True) if date else 'N/A'

# 3. 作者
author = soup.find('span', class_='kbwcm-author')
author_text = author.get_text(strip=True).rstrip(',') if author else 'N/A'

# 4. 正文内容（只取 <div class="detail-content"> 下所有段落文字）
content_div = soup.find('div', class_='detail-content')
if content_div:
    paragraphs = content_div.find_all('p')
    content_text = '\n'.join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
else:
    content_text = 'N/A'

# 5. 来源
source_text = url  # 原网页链接

# 写入 CSV
with open('kenh2025_article1.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['author', 'title', 'date', 'content', 'source'])
    writer.writeheader()
    writer.writerow({
        'author': author_text,
        'title': title_text,
        'date': date_text,
        'content': content_text,
        'source': source_text
    })

print(" 已保存为 kenh14_article.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# -------------------------------------------------------
#  Extraction pour l'article que tu as indiqué
# -------------------------------------------------------

url = "https://kenh14.vn/hoc-duong/ky-thi-thpt-quoc-gia-2015-lich-thi-cua-dh-quoc-gia-ha-noi-20150314080244768.chn"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.content, "html.parser")

# 1. Auteur
author_tag = soup.find('span', class_='kbwcm-author')
author_text = author_tag.get_text(strip=True).rstrip(',') if author_tag else ''

# 2. Titre
title_tag = soup.find('h1', class_='kbwc-title')
title_text = title_tag.get_text(strip=True) if title_tag else ''

# 3. Date
date_tag = soup.find('span', class_='kbwcm-time')
date_text = date_tag.get_text(strip=True) if date_tag else ''

# 4. Contenu principal
# On cible <div class="klw‑new‑content"> puis on récupère
# le chapeau sapo (optionnel) + tous les paragraphes dans <div class="knc-content">
content_list = []
sapo = soup.find('h2', class_='knc-sapo')
if sapo:
    sapo_text = sapo.get_text(strip=True)
    if sapo_text:
        content_list.append(sapo_text)

main_div = soup.find('div', class_='knc-content')
if main_div:
    for p in main_div.find_all('p'):
        text = p.get_text(strip=True)
        if text:
            content_list.append(text)

content_text = "\n\n".join(content_list)

# 5. URL de la page
page_url = url

# 6. Ecrire dans un CSV
with open('kenh2015_4.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['author', 'title', 'date', 'content', 'url'])
    writer.writeheader()
    writer.writerow({
        'author': author_text,
        'title': title_text,
        'date': date_text,
        'content': content_text,
        'url': page_url
    })

print(" Fichier kenh2015_.csv créé avec succès")


In [ ]:
#2016

import requests
from bs4 import BeautifulSoup
import csv

# -------------------------------------------------------
#  Extraction pour l'article que tu as indiqué
# -------------------------------------------------------

url = "https://kenh14.vn/top-3-hoa-hau-viet-nam-2022-nguoi-thang-giai-quoc-te-nguoi-hoc-len-thac-si-215241122144704936.chn"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.content, "html.parser")

# 1. Auteur
author_tag = soup.find('span', class_='kbwcm-author')
author_text = author_tag.get_text(strip=True).rstrip(',') if author_tag else ''

# 2. Titre
title_tag = soup.find('h1', class_='kbwc-title')
title_text = title_tag.get_text(strip=True) if title_tag else ''

# 3. Date
date_tag = soup.find('span', class_='kbwcm-time')
date_text = date_tag.get_text(strip=True) if date_tag else ''

# 4. Contenu principal
# On cible <div class="klw‑new‑content"> puis on récupère
# le chapeau sapo (optionnel) + tous les paragraphes dans <div class="knc-content">
content_list = []
sapo = soup.find('h2', class_='knc-sapo')
if sapo:
    sapo_text = sapo.get_text(strip=True)
    if sapo_text:
        content_list.append(sapo_text)

main_div = soup.find('div', class_='knc-content')
if main_div:
    for p in main_div.find_all('p'):
        text = p.get_text(strip=True)
        if text:
            content_list.append(text)

content_text = "\n\n".join(content_list)

# 5. URL de la page
page_url = url

# 6. Ecrire dans un CSV
with open('kenh2024_22.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['author', 'title', 'date', 'content', 'url'])
    writer.writeheader()
    writer.writerow({
        'author': author_text,
        'title': title_text,
        'date': date_text,
        'content': content_text,
        'url': page_url
    })

print(" Fichier kenh2015_.csv créé avec succès")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/kenh_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("kenh2024_"):
            zipf.write(os.path.join("/content", file), arcname=file)

In [ ]:
from google.colab import files
files.download("/content/kenh_all_csv.zip")


#2) https://cafebiz.vn


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网页
url = "https://cafebiz.vn/quoc-hoi-se-bau-chu-tich-nuoc-vao-thang-10-2024-17624082621173561.chn"
headers = {"User-Agent": "Mozilla/5.0"}

# 请求网页内容
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# 1. 标题
title_tag = soup.find("h1", class_="title")
title = title_tag.get_text(strip=True) if title_tag else ""

# 2. 日期
date_tag = soup.find("span", class_="time")
date = date_tag.get_text(strip=True) if date_tag else ""

# 3. 作者（优先尝试作者栏，再尝试来源栏）
author = ""
author_byline = soup.find("strong", class_="detail-author")
if author_byline and author_byline.get_text(strip=True):
    author = author_byline.get_text(strip=True).replace("Theo", "").strip()
else:
    author_tag = soup.find("span", class_="link-source-text-name")
    if author_tag:
        author = author_tag.get_text(strip=True)

# 4. 正文内容（只提取<p>和<blockquote>中的纯文本）
content_div = soup.find("div", class_="detail-content")
content = ""
if content_div:
    for tag in content_div.find_all(["p", "blockquote"]):
        text = tag.get_text(strip=True)
        if text:
            content += text + "\n"

# 5. 来源（当前URL）
source = url

# 6. 构建DataFrame并保存为CSV
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content.strip()],
    "source": [source]
}

df = pd.DataFrame(data)
df.to_csv("cafebiz2024_25.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 cafebiz2021_1.csv")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/cafebiz_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("cafebiz2024_"):
            zipf.write(os.path.join("/content", file), arcname=file)

In [ ]:
from google.colab import files
files.download("/content/cafebiz_all_csv.zip")


#3) https://genk.vn


#查询https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://genk.vn/ro-ri-ve-khung-long-rtx-5090-32gb-vram-gddr7-cuc-khung-hieu-nang-vuot-troi-nhung-ngon-dien-kinh-hoang-20250107002116059.chn"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# 标题
title_tag = soup.find("h1")
title = title_tag.get_text(strip=True) if title_tag else ""

# 日期
date_tag = soup.find("span", class_="kbwcm-time")
date = date_tag.get("title", "").strip() if date_tag else ""

# 作者
author_tag = soup.find("span", class_="kbwcm-author")
author = author_tag.get_text(strip=True).replace(",", "").strip() if author_tag else ""

# 正文内容
content = ""
# 第一段介绍：<h2 class="knc-sapo">
sapo = soup.find("h2", class_="knc-sapo")
if sapo:
    content += sapo.get_text(strip=True) + "\n"

# 正文段落 <div id="ContentDetail">
content_div = soup.find("div", id="ContentDetail")
if content_div:
    for tag in content_div.find_all("p"):
        text = tag.get_text(strip=True)
        if text:
            content += text + "\n"

# 来源
source = url

# 构建 DataFrame
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content.strip()],
    "source": [source]
}

# 导出为 CSV
df = pd.DataFrame(data)
df.to_csv("genk2025_23.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 genk_article.csv")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/genk_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("genk2025_"):
            zipf.write(os.path.join("/content", file), arcname=file)

In [ ]:
from google.colab import files
files.download("/content/genk_all_csv.zip")


#❌4) https://baomoi.com



#查询https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2



In [ ]:
#2015

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 要抓取的网址
url = "https://vtcnews.vn/ky-thi-thpt-quoc-gia-2015-cham-bai-thi-trac-nghiem-the-nao-ar189669.html"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# 标题（注意是 <h2 class="text-title">）
title_tag = soup.find("h2", class_="text-title")
title = title_tag.get_text(strip=True) if title_tag else ""

# 作者（注意是 <strong> 标签内，末尾可能重复一次）
author_tag = soup.find("strong")
author = author_tag.get_text(strip=True) if author_tag else ""

# 日期（<span class="time">）
date_tag = soup.find("span", class_="time")
date = date_tag.get_text(strip=True) if date_tag else ""

# 正文内容（在 <div class="detail-content afcbc-body" data-role="content">）
content = ""
content_div = soup.find("div", class_="detail-content")
if content_div:
    paragraphs = content_div.find_all("p", class_="body-text")
    for p in paragraphs:
        text = p.get_text(strip=True)
        if text:
            content += text + "\n"

# 构建 DataFrame
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content.strip()],
    "source": [url]
}

# 导出为 CSV
df = pd.DataFrame(data)
df.to_csv("baomoi2015_2.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 nguoiduatin_article.csv")


In [ ]:
#VTCNews



In [ ]:
#2025

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.nguoiduatin.vn/dap-an-de-thi-mon-tieng-anh-vao-lop-10-nam-2015-tai-ha-noi-204193589.htm"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# 标题
title_tag = soup.find("h1", class_="detail-title")
title = title_tag.get_text(strip=True) if title_tag else ""

# 作者
author_tag = soup.find("p", class_="name")
author = author_tag.get_text(strip=True) if author_tag else ""

# 日期
date_tag = soup.find("div", attrs={"data-role": "publishdate"})
date = date_tag.get_text(strip=True) if date_tag else ""

# 正文内容
content_div = soup.find("div", class_="detail-content")
content = ""

if content_div:
    paragraphs = content_div.find_all("p")
    for p in paragraphs:
        text = p.get_text(strip=True)
        if text:
            content += text + "\n"

# 构建 DataFrame
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content.strip()],
    "source": [url]
}

df = pd.DataFrame(data)
df.to_csv("baomoi2015_2.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 nld_article_20250807.csv")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/genk_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("genk2025_"):
            zipf.write(os.path.join("/content", file), arcname=file)

In [ ]:
from google.colab import files
files.download("/content/genk_all_csv.zip")


#❌ 5) https://vnexpress.net

#查询https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2





In [ ]:
#anti-scrapping

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 网址
url = "https://vnexpress.net/khoanh-khac-buon-cua-messi-thang-giai-anh-bao-chi-the-gioi-2014-3147342.html"
headers = {"User-Agent": "Mozilla/5.0"}

# 发送请求
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# 抓取标题
title_tag = soup.find("h1", class_="title-detail")
title = title_tag.get_text(strip=True) if title_tag else ""

# 抓取作者
author_tag = soup.find("strong")
author = author_tag.get_text(strip=True) if author_tag else ""

# 抓取时间
date_tag = soup.find("span", class_="date")
date = date_tag.get_text(strip=True) if date_tag else ""

# 抓取正文内容（包括摘要 + 正文段落）
content = ""

# 摘要段落
desc_tag = soup.find("p", class_="description")
if desc_tag:
    content += desc_tag.get_text(strip=True) + "\n"

# 主体段落
paragraphs = soup.find_all("p", class_="Normal")
for p in paragraphs:
    text = p.get_text(strip=True)
    if text:
        content += text + "\n"

# 构建 DataFrame
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content.strip()],
    "source": [url]
}

df = pd.DataFrame(data)

# 保存为 CSV 文件
df.to_csv("vnexpress_messi_2014.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 vnexpress_messi_2014.csv")


In [ ]:
# 安装 Chromium 和 ChromeDriver
!apt-get update > /dev/null
!apt-get install -y chromium chromium-driver > /dev/null

# 安装 Selenium
!pip install -q selenium


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# 配置 Chrome 选项（Headless + Colab 兼容）
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = "/usr/bin/chromium"

# 启动浏览器
driver = webdriver.Chrome(service=Service("/usr/lib/chromium/chromedriver"), options=chrome_options)

# 抓取网页
url = "https://vnexpress.net/khoanh-khac-buon-cua-messi-thang-giai-anh-bao-chi-the-gioi-2014-3147342.html"
driver.get(url)
time.sleep(3)  # 等待页面加载完毕

# 解析 HTML
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# 提取信息
title_tag = soup.find("h1", class_="title-detail")
title = title_tag.get_text(strip=True) if title_tag else ""

author_tag = soup.find("strong")
author = author_tag.get_text(strip=True) if author_tag else ""

date_tag = soup.find("span", class_="date")
date = date_tag.get_text(strip=True) if date_tag else ""

# 抓取正文内容（包括摘要 + 段落）
content = ""
desc_tag = soup.find("p", class_="description")
if desc_tag:
    content += desc_tag.get_text(strip=True) + "\n"

for p in soup.find_all("p", class_="Normal"):
    text = p.get_text(strip=True)
    if text:
        content += text + "\n"

# 构建 DataFrame
df = pd.DataFrame([{
    "author": author,
    "title": title,
    "date": date,
    "content": content.strip(),
    "source": url
}])

# 保存为 CSV
df.to_csv("vnexpress_messi_colab.csv", index=False, encoding="utf-8-sig")
print("抓取成功！文件名：vnexpress_messi_colab.csv")


In [ ]:
#下载包


import zipfile
import os

zip_path = "/content/vnexpress_all_csv.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir("/content"):
        if file.endswith(".csv") and file.startswith("vnexpress2015_"):
            zipf.write(os.path.join("/content", file), arcname=file)

In [ ]:
from google.colab import files
files.download("/content/genk_all_csv.zip")


#6) https://webtretho.com (forum)


#查询https://www.google.com/advanced_search?q=site:thepaper.cn+2015&client=firefox-b-d&sca_esv=4d1e52ecd0f51357&sxsrf=AE3TifOp3VsAjjic0H6D16Q26gmvMKT8dQ:1753550785198&tbas=0&biw=1232&bih=728&dpr=2





In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网址
url = "https://www.webtretho.com/f/tam-su-chuyen-cong-viec-cong-so/4-2015-tap-doan-nam-long-cong-bo-mo-ban-dot-cuoi-3-lo-dep-nhat-c5-c6-va-c7-nhan-nha-nam-2015-2036813"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# --------------------------
# 提取标题
title_tag = soup.find("h1", class_="text-wrap tw-font-bold text-primary tw-mt-2.5 tw-mb-1 tw-text-3xl")
title = title_tag.get_text(strip=True) if title_tag else ""

# 提取作者
author_tag = soup.find("a", class_="tw-font-bold text-main tw-text-xs")
author = author_tag.get_text(strip=True) if author_tag else ""

# 提取日期（注：这里是“几个月前”的形式）
date_tag = soup.find("div", class_="tw-text-xs text-gray-v2")
date = date_tag.get_text(strip=True) if date_tag else ""

# 提取正文内容：<div id="post-content">
content = ""
content_div = soup.find("div", id="post-content")
if content_div:
    for p in content_div.find_all("p", class_="editor-text-justify"):
        text = p.get_text(strip=True)
        if text:
            content += text + "\n"

# 构建 dataframe
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content.strip()],
    "source": [url]
}

df = pd.DataFrame(data)

# 保存为 CSV
df.to_csv("webtretho2015_2.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 webtretho_luat_2015.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 目标网址
url = "https://www.webtretho.com/f/hoc-tap-va-nghien-cuu/chia-se-phan-mem-dong-goi-installshield-2015"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# --------------------------
# 提取标题
title_tag = soup.find("h1", class_="text-wrap tw-font-bold text-primary tw-mt-2.5 tw-mb-1 tw-text-3xl")
title = title_tag.get_text(strip=True) if title_tag else ""

# 提取作者
author_tag = soup.find("a", class_="tw-font-bold text-main tw-text-xs")
author = author_tag.get_text(strip=True) if author_tag else ""

# 提取日期
date_tag = soup.find("div", class_="tw-text-xs text-gray-v2")
date = date_tag.get_text(strip=True) if date_tag else ""

# 提取正文内容（注意：这段文字不完全在 <p> 中）
content = ""
content_div = soup.find("div", id="post-content")
if content_div:
    # 提取所有文字节点（不管什么标签）
    content = content_div.get_text(separator="\n", strip=True)

# 构建 dataframe
data = {
    "author": [author],
    "title": [title],
    "date": [date],
    "content": [content],
    "source": [url]
}

df = pd.DataFrame(data)

# 保存为 CSV
df.to_csv("webtretho2015_5.csv", index=False, encoding="utf-8-sig")
print("抓取成功，已保存为 webtretho2015_2.csv")


In [ ]:
#Forum

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 标签页 URL
url = "https://www.webtretho.com/tags/rong-bien"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# 初始化数据列表
posts = []

# 遍历所有帖子容器
# 每个帖子是 <div role="article">...</div>
for article in soup.find_all("div", attrs={"role": "article"}):
    # 标题
    title_tag = article.find("h4")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # 作者
    author_tag = article.find("a", class_="tw-font-bold text-main tw-text-xs")
    author = author_tag.get_text(strip=True) if author_tag else ""

    # 日期
    date_tag = article.find("div", class_="tw-text-xs text-gray-blue tw-mt-1.5")
    date = date_tag.get_text(strip=True) if date_tag else ""

    # 内容（正文摘要）
    content_tag = article.find("div", class_="tw-leading-5 tw-text-sm text-black tw-whitespace-normal tw-break-words w-100 max-l-2")
    content = content_tag.get_text(separator="\n", strip=True) if content_tag else ""

    # 链接
    link_tag = title_tag.find_parent("a") if title_tag else None
    href = "https://www.webtretho.com" + link_tag["href"] if link_tag and link_tag.has_attr("href") else url

    # 添加一行数据
    posts.append({
        "title": title,
        "author": author,
        "date": date,
        "content": content,
        "link": href
    })

# 保存为 CSV
df = pd.DataFrame(posts)
df.to_csv("webtretho2015_6.csv", index=False, encoding="utf-8-sig")
print("已抓取", len(posts), "条帖子，保存为 webtretho_tag_rong_bien.csv")


In [ ]:
!apt-get update > /dev/null
!apt-get install -y chromium chromium-driver > /dev/null
!pip install -q selenium


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# 配置 headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = "/usr/bin/chromium"

# 初始化浏览器
driver = webdriver.Chrome(service=Service("/usr/lib/chromium/chromedriver"), options=chrome_options)

# 打开标签页
url = "https://www.webtretho.com/tags/rong-bien"
driver.get(url)
time.sleep(5)  # ⏳ 等待加载完成

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# 提取帖子内容
posts = []

for article in soup.find_all("div", attrs={"role": "article"}):
    # 标题
    title_tag = article.find("h4")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # 作者
    author_tag = article.find("a", class_="tw-font-bold text-main tw-text-xs")
    author = author_tag.get_text(strip=True) if author_tag else ""

    # 日期
    date_tag = article.find("div", class_="tw-text-xs text-gray-blue tw-mt-1.5")
    date = date_tag.get_text(strip=True) if date_tag else ""

    # 内容（正文摘要）
    content_tag = article.find("div", class_="tw-leading-5 tw-text-sm text-black tw-whitespace-normal tw-break-words w-100 max-l-2")
    content = content_tag.get_text(separator="\n", strip=True) if content_tag else ""

    # 链接
    link_tag = title_tag.find_parent("a") if title_tag else None
    href = "https://www.webtretho.com" + link_tag["href"] if link_tag and link_tag.has_attr("href") else url

    posts.append({
        "title": title,
        "author": author,
        "date": date,
        "content": content,
        "link": href
    })

# 保存为 CSV
df = pd.DataFrame(posts)
df.to_csv("webtretho_tag_rong_bien.csv", index=False, encoding="utf-8-sig")
print(f"成功抓取 {len(posts)} 条内容，已保存为 webtretho_tag_rong_bien.csv")
